In [1]:
from nltk.corpus import movie_reviews as mr, stopwords
import nltk
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('movie_reviews')
nltk.download('stopwords')
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import re

[nltk_data] Downloading package movie_reviews to C:\Users\Noel
[nltk_data]     Nathan\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Noel
[nltk_data]     Nathan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



Función que se utiliza para limpiar palabras, si es False, no se utiliza esa palabra

In [2]:
def clean_words(word):
    word = re.sub('_', ' ', word)
    if word in stopwords.words('english'):
        return False
    
    if re.search(r'\d+', word): # Remove numbers
        return False
    
    
    # use only words in the english dictionary
    # if not WordNetLemmatizer().lemmatize(word).lower() in nltk.corpus.words.words():
    #     return False
    
    return True

Entrenamos el CountVectorizer

In [4]:

# Text tokenizated
words = mr.words()


# Apply lemmatization to each word
lemmatizer = WordNetLemmatizer()
lemmas = [lemmatizer.lemmatize(word) for word in words]

# Remove stop words from the lemmas
# stop_words = stopwords.words('english')
# filtered_lemmas = filter(lambda lemma: lemma not in stop_words, lemmas)
filtered_lemmas = filter(clean_words, lemmas)

# Apply count vectorization to the filtered lemmas
vectorizer = CountVectorizer()
count_vectors = vectorizer.fit(filtered_lemmas)

En reviews se carga cada texto con su categoria -> ('victor és un troç d'espinaca', 'negativo')

In [5]:
from nltk.corpus import movie_reviews

reviews = [(movie_reviews.raw(fileid), category)
           for category in movie_reviews.categories()
           for fileid in movie_reviews.fileids(category)]

# Convert the reviews into a pandas DataFrame
df = pd.DataFrame(reviews, columns=['review', 'sentiment'])

# Map the sentiment labels to binary values (0 for negative and 1 for positive)
df['sentiment'] = df['sentiment'].map({'neg': 0, 'pos': 1})

# Print the first few rows of the DataFrame to check the output
print(df.head())

                                              review  sentiment
0  plot : two teen couples go to a church party ,...          0
1  the happy bastard's quick movie review \ndamn ...          0
2  it is movies like these that make a jaded movi...          0
3   " quest for camelot " is warner bros . ' firs...          0
4  synopsis : a mentally unstable man undergoing ...          0


Aplicamos a los datos el count vectorize

In [6]:
count_vectorized = vectorizer.transform(df['review'])

In [7]:
count_array = count_vectorized.toarray()
test = pd.DataFrame(data=count_vectorized.toarray(),columns = vectorizer.get_feature_names_out())
test.to_csv('test.csv', sep=';')

In [8]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X = vectorizer.transform(df['review'])
y = df['sentiment']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Creating models

In [9]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

param_grid = {
    'alpha': [0.1, 1.0, 10.0],
    'fit_prior': [True, False]
}

nb = MultinomialNB()
grid_search = GridSearchCV(nb, param_grid, cv=5)

grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)


Best parameters: {'alpha': 1.0, 'fit_prior': True}
Best score: 0.804375


In [10]:
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [100, 200, 500],
    'max_features': ['sqrt', 'log2'],
    'max_depth' : [4,6,8],
    'criterion' :['gini', 'entropy']
}

rfc = RandomForestClassifier()

grid_search = GridSearchCV(rfc, param_grid, cv=5)

grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)


Best parameters: {'criterion': 'entropy', 'max_depth': 6, 'max_features': 'sqrt', 'n_estimators': 500}
Best score: 0.8450000000000001


In [11]:
import xgboost as xgb
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'n_estimators': [50, 100, 200],
    'gamma': [0, 0.1, 0.2]
}

xgb_model = xgb.XGBClassifier()

grid_search = GridSearchCV(xgb_model, param_grid, cv=5)

grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

Best parameters: {'gamma': 0, 'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 200}
Best score: 0.8125
